# Part 2: Simple model

### Task 0: Grouping labels

See 'group_data.ipynb' file for implementation.

### Task 1: Logistic Regression model

In [ ]:
import pandas as pd

# load grouped training data
src = 'data/training_data_grouped.csv'
train_data_grp = pd.read_csv(src)

# load grouped validation data
src = 'data/validation_data_grouped.csv'
train_data_grp = pd.read_csv(src)

In [ ]:
# Code reference: https://www.freecodecamp.org/news/how-to-build-and-train-linear-and-logistic-regression-ml-models-in-python/

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# copy data
model_data = group_data.copy(deep=True)

# Make grouped types ('fake' or 'reliable') into 'true' or 'false' values
type_data = pd.get_dummies(model_data['type'], drop_first=True)
model_data = pd.concat([model_data, type_data], axis=1)

# remove columns
model_data.drop(['type',
               'content',
               'title',
               'authors',
               'content_clean', 
               'content_stopword',
               'content_stem'
               ], axis = 1, inplace = True)

# Split data into training and prediction data
y_data = model_data['reliable']
x_data = model_data.drop(['reliable'], axis=1)

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x_data, y_data, test_size = 0.1)

# create logistic reg. model, and train it
model = LogisticRegression()
model.fit(x_training_data, y_training_data)

# test the model and report performance
predictions = model.predict(x_test_data)
print(classification_report(y_test_data, predictions))